<center> <h1> Python Web Navigation </center> </h1>
<center> <h3> HireRite Grand Challege </center> </h3>
<center> <h5> Matt Raymond </center> </h5>

# Imports

In [1]:
# Web driver for comtroling chrome
from selenium import webdriver

# Regex for parsing website navigation
import re as regx

# Thread sleeping
from time import sleep

# Import for dates
from datetime import datetime

# Import keys so we can press enter
from selenium.webdriver.common.keys import Keys

# Wait function so the webpage an finish loading
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from parse import ImageParsing

from selenium.webdriver.chrome.options import Options

import math
import tempfile
import os
from PIL import Image


# Browser

In [2]:
# Sets of the path of the chrome driver
# ToDO: Fix the chrome driver path so it doesn't need to be set automatically inside python
__BROWSER__ = webdriver.Chrome(executable_path='/home/trevor/Documents/HireRightChallenge/test_reassembly/chromedriver_linux64/chromedriver')

# Set the implicit wait time so that webpages don't flip out when they can't fontrol things right away
__BROWSER__.implicitly_wait(5)

__PARSING__ = ImageParsing()

# Global Testing Variables

In [3]:
# The websites that we're supposed to use
__TESTINGWEBSITES__ = ["https://www.sccourts.org/casesearch/", # agreement says no crawlers
                       "https://wcca.wicourts.gov/case.html", # added captcha
                       "https://www.oscn.net/dockets/search.aspx",
                       "https://www.civitekflorida.com/ocrs/county/", # recapcha
                       "https://apps.supremecourt.az.gov/publicaccess/caselookup.aspx", # captcha
                       "https://www.superiorcourt.maricopa.gov/docket/CriminalCourtCases/caseSearch.asp",
                       "http://justicecourts.maricopa.gov/FindACase/casehistory.aspx",
                       "https://casesearch.epcounty.com/PublicAccess/default.aspx"]

# the websites that are actually LEGAL to use
__LEGALWEBSITES__ = __TESTINGWEBSITES__[2:3] + __TESTINGWEBSITES__[-3:]

# Testing variables for name and date of birth
__TESTING_FNAME__ = "John"
__TESTING_LNAME__ = "Smith"
__TESTING_MNAME__ = "Herbert"
__TESTING_DOB__ = "09-11-2001"

# Regex

In [4]:
# Regex for different searches
__FIRSTNAME__ = "=\"[^\"]*[Ff][A-Za-z]*[Nn]ame[A-Za-z]*\""
__LASTNAME__ = "=\"[^\"]*[Ll][A-Za-z]*[Nn]ame[A-Za-z]*\""
__MIDDLENAME__ = "=\"[^\"]*[Mm][A-Za-z]*[Nn]ame[A-Za-z]*\""
__BIRTHDATE__ = "=\"[^\"]*?(search)?[^\"]*d(ate)?of?b(irth)?[^\"^(search)]*?\""
__DATEFORMAT__ = "[MD\d]{2}[./-][MD\d]{2}[./-][Y\d]{4}"
__REACTFORMAT__ = "<div id=\"[^\"]*react[^\"]*\"><\/div>"
__OTHER_LOADING_FORMAT__ = "svgclippaths\">\<head>\<meta"
__SEARCHBUTTON__ = "=\"[^\"]*?((?<!menu)(?<!re)(?<!box)search)[^\"]*?\""
__CASELINK__ = "(?<=href=\")([^\"]*((number)|(case((no)|(id))))[^\"]*)(?=\")"
__NEXT_PAGE_ID__ = "(?<=id=\")[^\"]*next(?=\")"
__NEXT_PAGE_HREF__ = "(?<=href=\")[^\"]*(?=\">next)"
__URL_PAGE_NUMBER__ = "(?<=page=)\d*(?=&)"
__COUNTY__ = "(([A-Z][a-zA-Z\-]*\ )|([a-zA-Z\-]*(&nbsp;)+)){1,3}[Cc][Oo][Uu][Nn][Tt][Yy]"

# Function Declaration

## Scraping Info from Webpage

In [5]:
'''
Scrapes the url from a webpage
Input: takes a string strURL as a url to a webpage
Output: returns the html from the webpage as a string
'''
def getSource(strURL):
    __BROWSER__.get(strURL)
    print("Scraping website source code...")
    return pauseforReact(__BROWSER__.page_source)

'''
Waits for react elements to load so that we have all of the data loaded before scraping
Input: takes a string html from the webpage to check for react elements
Output: returns the final html source as a string
'''
def pauseforReact(html):
    while regx.search(__REACTFORMAT__, html, regx.IGNORECASE) is not None and regx.search(__OTHER_LOADING_FORMAT__, html, regx.IGNORECASE) is None:
        sleep(1)
        html = __BROWSER__.page_source
    return __BROWSER__.page_source

'''
Searches for the name of an element given a regex description
Input: takes a string ws as the source html and a string re as the regex search criteria
Output: returns the name of the element found as a string
'''
def regSearchName(ws, re):
    # Searches for a tag with a given name
    _result = regx.search("name" + re, ws, regx.IGNORECASE)
    if(_result is None):
        return "";
    else:
        # Return the result minus the "name=" part
        return _result[0][6:-1]

'''
Searches for the ID of an element given a regex description
Input: takes a string ws as the source html and a string re as the regex search criteria
Output: returns the name of the element found as a string
'''
def regSearchID(ws, re):
    _result = regx.search("id" + re, ws, regx.IGNORECASE)
    if(_result is None):
        return "";
    else:
        return _result[0][4:-1]

'''
Finds the id and name of the object that meets a certain search criteria
Input: takes a string ws as the source html and a string re as the regex search criteria
Output: returns the id and name of the located element as a tuple
  - This allows for our logic to include cases where the name field is filled and the id isn't or vice versa
'''
def getIDandName(ws, re):
    return (regSearchID(ws, re), regSearchName(ws, re))

'''
Finds the county name on the page
Input: takes a string html as the page html
Output: returns a string for the county
'''
def findCountyName(html):
    states = ["alabama", "alaska", "arizona", "arkansas", "california", "colorado", "connecticut", "delaware",
        "florida", "georgia", "hawaii", "idaho", "illinois", "indiana", "iowa", "kansas", "kentucky",
        "louisiana", "maine", "maryland", "massachusetts", "michigan", "minnesota", "mississippi", "missouri",
        "montana", "nebraska", "nevada", "new hampshire", "new jersey", "new mexico", "new york",
        "north carolina", "north dakota", "ohio", "oklahoma", "oregon", "pennsylvania", "rhode island",
        "south carolina", "south dakota", "tennessee", "texas", "utah", "vermont", "virginia", "washington",
        "west Virginia", "wisconsin", "wyoming"]
    
    _result = regx.search(__COUNTY__, html)
    if(_result):
        _result = _result[0].replace("&nbsp;", " ").lower()
        
        # Removes the state name from the title if it's included
        for i in states:
            _result = _result.replace(i+" ", "")
        return _result
    else:
        return None

## Input Filling

In [6]:
'''
Fills out the person's name on the website
Input: takes a string html as the website's source html, string fn as the first name,
       a string ln as the last name, and a string mn as the middle name
Output: None
'''
def fillOutName(html, fn, ln, mn):
    # Finds the ID and Name for first, middle, and last names
    fnID, fnName = getIDandName(html, __FIRSTNAME__)
    mnID, mnName = getIDandName(html, __MIDDLENAME__)
    lnID, lnName = getIDandName(html, __LASTNAME__)

    sendInput(fnName, fn)
    sendInput(mnName, mn)
    sendInput(lnName, ln)

'''
Sends input to an element
Input: takes a string e as a web element name and a string s as the text to be sent
Output: None
Exception: prints an error message saying what text it tried to enter into what element
'''
def sendInput(e, s):
    try:
        # Breaks out if there's no element sent
        if str(e) is "":
            return
        
        # Find the element based on the name and send input
        item = __BROWSER__.find_element_by_name(e)
        item.click()
        item.clear()
        item.send_keys(s)
        
    except:
        print("Input failed. Attempted to enter \"{0}\" to \"{1}\" and failed.".format(s, e))
    
# Fill out the date of birth
# Returns a bool denoting whether it was successful or not

'''
Fills out the date of birth on the page
Input: takes a string html as the website source html and a string dob
Output: returns a bool saying whether it was successful or not
Exception: prints a message saying that it failed and what it tried to enter as a dob
'''
def fillOutDOB(html, dob):
    try:
        # If there is a dob given
        if dob is not None:
            # get the id and name of the date-picker element
            dID, dName = getIDandName(html, __BIRTHDATE__)
        
            # Try to find the element by name because this is more common
            if dName is not "":
                item = __BROWSER__.find_element_by_name(dName)
            # If it can't find it, try by id
            else:
                item = __BROWSER__.find_element_by_id(dID)
            
            # If the item is locked then there's no poin in trying to edit it
            if not item.is_enabled():
                return False
            
            # Select the element and remove all of the information already in it
            item.click()
            item.clear()
            
            # Find the delimiter that the webpage wants us to use (in case it's picky)
            _delin = findDateFormat(html)
            
            # Create a date based on the american standard (since these are american websites),
            # using the specified delimiters
            _date = "{1}{0}{2}{0}{3}".format(_delin, str(dob.month).zfill(2), str(dob.day).zfill(2), dob.year)
            
            # Fill out the date
            item.send_keys(_date)
        return True
    
    # If something goes wrong and the date input can't be found or can't be edited
    except:
        print("Input failed. Attempted to enter \"{0}\" as a date and failed.".format(dob))
        return False
        
# Find the delimiters used in the example (if one exists) and returns them

'''
Finds the date format being used on the page. This is to ensure that the crawler doesn't
get stuck because of overly-picky websites
Input: takes a string html as the website's source html
Output: returns a string as either the found delimeter, or a dash as the default
'''
def findDateFormat(html):
    # Find the example date on the page
    result = regx.search(__DATEFORMAT__, html, regx.IGNORECASE)
    
    # Return either the result or use a random one that I happened to pick
    if result is not None:
        return result[0][2]
    else:
        return '-'

# Initialize the search

'''
Submits the search criteria
Input: takes a string html as the website source html
Output: None
'''
def search(html):
    # Saves the url for a comparison later
    lastURL = __BROWSER__.current_url
    
    # Gets the id and name of the first name field
    fnID, fnName = getIDandName(html, __FIRSTNAME__)
    
    # Tries pressing enter to submit
    # Some websites have search buttons that are really difficult to find,
    # so this makes it a lot easier
    __BROWSER__.find_element_by_name(fnName).send_keys(Keys.ENTER)
    
    # If we didn't redirect, then we need to actually click the search button
    if lastURL == __BROWSER__.current_url :
        # Gets the id and name of the search button
        sID, sName = getIDandName(html, __SEARCHBUTTON__)

        # If there is a search button within a reasonable distance
        if (sID is not "" or sName is not "") and (sID in html or sName in html):
            try:
                # Find and click the button
                item = __BROWSER__.find_element_by_name(sName)
                item.click()
            except:
                return
    
'''
Fills out all of the information given
Input: takes in a stirng html as the website source html, a string fn as the first name,
       a string ln as the last name, a string mn as the middle name (defaulted to en empty string),
       and a date dob as the date of birth (defaulted to None)
Output: None
'''
def fillOutInformation(html, fn, ln, mn = "", dob = None):
    # NOTE: The fields must be filled out in this order or sometimes
    # the open date selector messes with the submission
    
    # Variable that checks whether the dob was filled out correctly
    dobFilled = fillOutDOB(html, dob)
    
    fillOutName(html, fn, ln, mn)
    
    # Sometimes the dob section is locked until you enter a name,
    # so this checks to see if the date of birth was filled out or not
    if not dobFilled:
        fillOutDOB(html, dob)

'''
Formats a html url to correctly format special characters
Input: takes a string s as a url
Output: returns a string as a url
'''
def formatURL(s):
    return s.replace("&amp;", "&")

# Testing

In [7]:
'''
Cycles through all of the cases on a page, opens the links, and returns back to the page at the end
Input: takes a string currentURL as the url of the page currently active, and an int numberOfTimes
       as a testing variable that says how many links should be run through. It is defaulted to -1,
       which is infinite
Output: None
'''
def cycleThroughCases(currentURL, landingCounty, numberOfTimes = -1):
    # Waits intil the results page has actually loaded
    while __BROWSER__.current_url == currentURL:
        sleep(1)
    
    # Waits for all of the react elements to load
    html = pauseforReact(__BROWSER__.page_source)
    
    # Find all of the links to cases
    results = list(regx.findall(__CASELINK__, html, regx.IGNORECASE))
    
    # save the current url so we can redirect back
    currentURL = __BROWSER__.current_url
    
    # Not sure why, but sometimes a tuple is returned and we have to take that into account
    isTuple = type(results[0]) is tuple
    
    # VIterator variable for comparing against numberOfTimes
    i = 0
    
    # Selects the first element if it's in a tuple
    if isTuple:
        results = [r[0] for r in results]
            
    # Removes duplicate links
    results = list(dict.fromkeys(results))
        
    searchCounty = findCountyName(html)
        
    # Cycles through all results
    for r in results:
        i += 1
                    
        # Skip over links to css files
        if ".css" not in r:
            print("Scraping {0}".format(r))
            
            try:
            
                # Find the element based on the link
                __BROWSER__.execute_script("window.open(\"" + formatURL(r) + "\",\"_blank\")")
                __BROWSER__.switch_to.window(__BROWSER__.window_handles[1])
                
                resultCounty = findCountyName(html)
                
                lstCounties = [landingCounty, searchCounty, resultCounty]
                
                # ThIs Is WhErE tHe WeBsCrApInG sHoUlD gO
                # Pass in lstCounty
                imgString = save_fullpage_screenshot(__BROWSER__, "./Screenshots/tmp.png")
                print(ParseImage(imgString, lstCounties))
                

                sleep(2) # Debug: So we can see the page

                # Redirect back to the page
                __BROWSER__.close()
                __BROWSER__.switch_to.window(__BROWSER__.window_handles[0])
                sleep(2) # Debug: So we can see the page
            except Exception as e:
                print(e)
            
            # Breaks for testing purposes
            if(i == numberOfTimes):
                break

In [8]:
'''
Figures out if there's a next page and switches to it if there is
Input: takes a bool sc that tells whether we should click (true) or whether
       we should edit the url (false)
Output: returns a bool that tells whether the page can redirect again (true)
        or whether this is the last page (false)
'''
def findNextPage(sc):
    # Saves the html of the current page
    currentHTML = __BROWSER__.page_source
    
    # If we're going the clicking route
    if sc:
        # Find all of the elements that can be used to get to the next page
        _result = regx.findall(__NEXT_PAGE_ID__, currentHTML, regx.IGNORECASE)
        
        # If that returns anything
        if len(_result) is not 0:
            # Find the first element
            nextButton = __BROWSER__.find_element_by_id(_result[0])
            
            # Check to see if the element is disabled
            returnValue = ("disabled" not in nextButton.get_attribute("class"))
            
            # Click 'cause why not
            nextButton.click()
            
            # Return whether it was disabled
            return returnValue
        
        # If we're going to click a link 
        else:
            # Find the links
            _result = regx.findall(__NEXT_PAGE_HREF__, currentHTML, regx.IGNORECASE)

            # If there are any
            if len(_result) is not 0:
                # Find the element and click it
                __BROWSER__.find_element_by_xpath('//a[@href="'+formatURL(_result[0])+'"]').click()
                return True
            # Otherwise return false
            return False
    else:
        return False

In [9]:
# Cycles through all pages

'''
Cycles through all pages and scrapes all of them
Input: takes an int noOfPages which tells how many pages to cycle through (for debugging).
       The default is -1, which means infinite
Output: None
'''
def cycleThroughAllPages(landingCounty, noOfPages = -1):
    
    html = __BROWSER__.page_source
    
    # Find whether there is a link for the next page or not 
    _result = regx.findall(__NEXT_PAGE_HREF__, html, regx.IGNORECASE)
        
    # If there's a link, then you should click it
    shouldClick = (len(_result) > 0)
    
    url = ""
    
    # While we're still supposed to continue
    while noOfPages != 0:
        
        cycleThroughCases(url, landingCounty, 5) ##randomthingy
        url = __BROWSER__.current_url
        # If we're not supposed to continue, then break
        if not findNextPage(shouldClick):
            break;

#         sleep(2) # Debugging 
        noOfPages -= 1

In [10]:
def navigateElPaso():
    if("epcounty" in __BROWSER__.current_url):
        __BROWSER__.find_elements_by_class_name("ssSearchHyperLink")[0].click()
        __BROWSER__.find_element_by_xpath("//select[@name='SearchBy']/option[text()='Defendant']").click()

In [11]:
def scanWebsite(ws, fn, ln, mn = "", dob = ""):
    __BROWSER__.switch_to.window(__BROWSER__.window_handles[0])
    __BROWSER__.get(ws)
    navigateElPaso()
    html = __BROWSER__.page_source
    
    try:
        convertedDate = datetime.strptime(dob, '%m-%d-%Y')
    except:
        convertedDate = None
     
    fillOutInformation(html, fn, ln, mn, convertedDate)
    search(html)
    try:
        landingCounty = findCountyName(html)
        cycleThroughAllPages(landingCounty, 2)
    except:
        print("There were no cases for this person")
    
def ParseImage(strFilepath, lstCounties):
    return __PARSING__.parseData(strFilepath, lstCounties)

In [12]:
# Adapted from https://arthurpemberton.com/2017/12/full-page-screenshots-with-python-and-selenium

def save_fullpage_screenshot(driver, output_path, tmp_prefix='selenium_screenshot', tmp_suffix='.png'):
    """
    Creates a full page screenshot using a selenium driver by scrolling and taking multiple screenshots,
    and stitching them into a single image.
    """
 
#     # get the page
#     driver.get(url)
 
    # get dimensions
    window_height = driver.execute_script('return window.innerHeight')
    scroll_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    num = int( math.ceil( float(scroll_height) / float(window_height) ) )
 
    # get temp files
    tempfiles = []
    for i in range( num ):
        fd,path = tempfile.mkstemp(prefix='{0}-{1:02}-'.format(tmp_prefix, i+1), suffix=tmp_suffix)
        os.close(fd)
        tempfiles.append(path)
        pass
 
    try:
        # take screenshots
        for i,path in enumerate(tempfiles):
            if i > 0:
                driver.execute_script( 'window.scrollBy(%d,%d)' % (0, window_height) )
             
            driver.save_screenshot(path)
            pass
         
        # stitch images together
        stiched = None
        for i,path in enumerate(tempfiles):
            img = Image.open(path)
             
            w, h = img.size
            y = i * window_height
             
            if i == ( len(tempfiles) - 1 ):
                img = img.crop((0, h-(scroll_height % h), w, h))
                w, h = img.size
                pass
             
            if stiched is None:
                stiched = Image.new('RGB', (w, scroll_height))
             
            stiched.paste(img, (
                0, # x0
                y, # y0
                w, # x1
                y + h # y1
            ))
            pass
        stiched.save(output_path)
    finally:
        # cleanup
        for path in tempfiles:
            if os.path.isfile(path):
                os.remove(path)
        pass
 
    return output_path

In [17]:
scanWebsite(__LEGALWEBSITES__[0], __TESTING_FNAME__, __TESTING_LNAME__, "", "")

IndexError: list index out of range